In [1]:
import pandas as pd
import nltk
import hashlib
import json
import numpy as np

In [2]:
def kgrams_hash(sentence, k=5):
    hashes = []
    words = nltk.word_tokenize(sentence)
    for i in range(0, len(words) - k + 1):
        mystring = ' '.join(words[i:i+k])
        hash_object = hashlib.md5(mystring.encode())
        hsh = hash_object.hexdigest()
        hashes.append(hsh)
    return hashes        

In [3]:
def winnow(text, k=5, w=6):
    fingerprints = set([])
    sentences = nltk.sent_tokenize(text)
    dic = {}
    for index, sentence in enumerate(sentences):
        #if sentence
        sentence_hashes = kgrams_hash(sentence, k)
#         print (sentence_hashes)
        if len(sentence_hashes) and len(sentence_hashes) < w:
#             print ("small")
            fingerprints.add(min(sentence_hashes))
#             dic[min(sentence_hashes)] = index
        else:
            for i in range(0, len(sentence_hashes) - w + 1):
                window = sentence_hashes[i:i+w]
                fingerprints.add(min(window))
#                 dic[min(window)] = index
#                 print (window)
#     return sentences, dic, fingerprints
    return fingerprints

In [4]:
df = pd.read_pickle('plg/data_frames/Electronics_5.pkl')

In [5]:
df.head()

,index,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,1,0528881469,"[12, 15]",1,"I'm a professional OTR truck driver, and I bou...","11 25, 2010",AMO214LNFCEI4,Amazon Customer,Very Disappointed,1290643200
1,2,0528881469,"[43, 45]",3,"Well, what can I say. I've had this unit in m...","09 9, 2010",A3N7T0DY83Y4IG,C. A. Freeman,1st impression,1283990400
2,56,0972683275,"[15, 19]",5,Quality was excellent. Instructions were clear...,"10 5, 2010",A3QH8VQDE7HZCR,costaricachris,Real value for the money,1286236800
3,82,0972683275,"[8, 18]",1,I checked around Amazon as well as some other ...,"12 1, 2010",A38FGQVJM18OWV,"George S. Mitchell ""gsmitchell""",What a piece of junk!,1291161600
4,141,0972683275,"[14, 19]",5,"I mounted this in my RV and it holds a 26"" LCD...","09 30, 2010",A7R4R9THNELTP,LG,"Excellent, at any price",1285804800


In [6]:
min_helpfull_review = 10
helpfulness = df['helpful']
temp = [i for i in range(df.shape[0])  if helpfulness[i][1] > min_helpfull_review]
df_temp = df.iloc[temp].reset_index()
df = df_temp

In [7]:
num_reviews = df.shape[0]
print (df.shape)

(97747, 11)


In [8]:
reviews = df['reviewText']

In [9]:
fingerprints = []
curr_review = reviews[0:num_reviews]
for index, rev in enumerate(curr_review):
    if (index % 1000 == 0):
        print ("Fingerprints Computed: ", index)
    fingerprints.append(winnow(rev))

Fingerprints Computed:  0
Fingerprints Computed:  1000
Fingerprints Computed:  2000
Fingerprints Computed:  3000
Fingerprints Computed:  4000
Fingerprints Computed:  5000
Fingerprints Computed:  6000
Fingerprints Computed:  7000
Fingerprints Computed:  8000
Fingerprints Computed:  9000
Fingerprints Computed:  10000
Fingerprints Computed:  11000
Fingerprints Computed:  12000
Fingerprints Computed:  13000
Fingerprints Computed:  14000
Fingerprints Computed:  15000
Fingerprints Computed:  16000
Fingerprints Computed:  17000
Fingerprints Computed:  18000
Fingerprints Computed:  19000
Fingerprints Computed:  20000
Fingerprints Computed:  21000
Fingerprints Computed:  22000
Fingerprints Computed:  23000
Fingerprints Computed:  24000
Fingerprints Computed:  25000
Fingerprints Computed:  26000
Fingerprints Computed:  27000
Fingerprints Computed:  28000
Fingerprints Computed:  29000
Fingerprints Computed:  30000
Fingerprints Computed:  31000
Fingerprints Computed:  32000
Fingerprints Computed: 

In [10]:
plg_dic = {}
for i in range(num_reviews):
    if (i % 10000 == 0):
        print ("Reviews Parsed: ", i)
    for fp in fingerprints[i]:
        if fp in plg_dic:
            indices = plg_dic[fp]
            indices.append(i)
            plg_dic[fp] = indices
        else:
            plg_dic[fp] = [i]

Reviews Parsed:  0
Reviews Parsed:  10000
Reviews Parsed:  20000
Reviews Parsed:  30000
Reviews Parsed:  40000
Reviews Parsed:  50000
Reviews Parsed:  60000
Reviews Parsed:  70000
Reviews Parsed:  80000
Reviews Parsed:  90000


In [11]:
plg_indices = []
for items in plg_dic:
    if (len(plg_dic[items]) > 1):
        temp = plg_dic[items]
        sz = len(temp)
        for i in range(sz):
            for j in range(i + 1, sz):
                plg_indices.append((temp[i], temp[j]))

In [35]:
plg_indices = set(plg_indices)

In [36]:
len(plg_indices)


6050345

In [38]:
scores = []
sentence_threshold = 2 #minimum number of windows
for index, items in enumerate(plg_indices):
    if (index % 100000 == 0):
        print (index)
    i = items[0]
    j = items[1]
    common = fingerprints[i] & fingerprints[j]
    temp = np.sqrt(len(fingerprints[i]) * len(fingerprints[j]))
    score = len(common)/temp
    if score > 0.7 and temp >= sentence_threshold:
        scores.append((i, j, score))

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
13

In [40]:
len(scores)

991

In [41]:
uni_scores = []
for index, item in enumerate(scores):
    i = item[0]
    j = item[1]
    if df.iloc[i]['asin'] != df.iloc[j]['asin']:
        uni_scores.append(item)

In [42]:
len(scores)

991

In [43]:
len(uni_scores)

988

In [44]:
array = df[['overall', 'asin', 'helpful']].values

In [45]:
df.iloc[69320]

level_0                                                       69320
index                                                         83697
asin                                                     B00562O8U0
helpful                                                    [89, 92]
overall                                                           4
reviewText        I recently bought a Sony Bravia KDL-55NX720 wh...
reviewTime                                              04 20, 2012
reviewerID                                           A1MF4WLGZDBP5K
reviewerName                                             P. Carlson
summary                Comparison of Several Active Shutter Glasses
unixReviewTime                                           1334880000
Name: 69320, dtype: object

In [46]:
df.iloc[66937]

level_0                                                       66937
index                                                         30077
asin                                                     B004UAU7CC
helpful                                                  [111, 111]
overall                                                           3
reviewText        I recently bought a Sony Bravia KDL-55NX720 wh...
reviewTime                                               04 6, 2012
reviewerID                                           A1MF4WLGZDBP5K
reviewerName                                             P. Carlson
summary           Comparison of 3 Different 3D Sony Compatible g...
unixReviewTime                                           1333670400
Name: 66937, dtype: object

In [47]:
item = array[0][1]
prev_item = 'null'
count = 0
X_all = [] # stores the deviation of each item
y_all = []
y = []
num_items = 0
sum = 0
item_array = np.array([])

for i in range(num_reviews):
    count = count + 1
    if (count % 10000 == 0):
        print (count)
    curr_item = array[i]
    item = curr_item[1]
    if (prev_item == item):
        item_array = np.append(item_array, curr_item[0])
        y.append(curr_item[2][0] / curr_item[2][1])
        sum += curr_item[0]
        if (count == num_reviews):
            mean = np.mean(item_array)
            item_array = np.rint((item_array - mean) * 2) / 2
            X_all = X_all + item_array.tolist()
            y_all = y_all + y
    else:
        if (num_items):
                mean = np.mean(item_array)
                item_array = np.rint((item_array - mean) * 2) / 2
                X_all = X_all + item_array.tolist()
                y_all = y_all + y

        num_items += 1
        sum = 0
        item_array = np.array([])
        y = []
        prev_item = item
        item_array = np.append(item_array, curr_item[0])
        y.append(curr_item[2][0] / curr_item[2][1])
        sum += curr_item[0]

10000
20000
30000
40000
50000
60000
70000
80000
90000


In [56]:
great_table = np.zeros([9, 9], dtype=int)
less_table = np.zeros([9, 9], dtype=int)
count = 0
threshold = 0.2
for index, item in enumerate(uni_scores):
    i = item[0]
    j = item[1]
    review1 = df.iloc[i]
    review2 = df.iloc[j]
    deviation1 = abs(X_all[i])
    deviation2 = abs(X_all[j])
    helpfulness1 = y_all[i]
    helpfulness2 = y_all[j]
    if deviation1 < deviation2:
        if helpfulness1 - helpfulness2 > threshold:
            great_table[int(2*deviation1)][int(2*deviation2)] += 1
        if helpfulness1 - helpfulness2 < -1 * threshold:
            less_table[int(2*deviation1)][int(2*deviation2)] += 1
    if deviation2 < deviation1:
        if helpfulness2 - helpfulness1 > threshold:
            great_table[int(2*deviation2)][[int(2*deviation1)]] += 1
        if helpfulness2 - helpfulness1 < -1 * threshold:
            less_table[int(2*deviation2)][[int(2*deviation1)]] += 1
print (great_table)
print (less_table)

[[0 2 4 3 2 4 0 1 0]
 [0 0 4 3 0 1 0 0 0]
 [0 0 0 4 1 0 1 0 0]
 [0 0 0 0 1 1 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 3 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]
[[0 2 1 1 1 1 0 0 0]
 [0 0 3 0 0 0 0 0 0]
 [0 0 0 1 1 0 0 0 0]
 [0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]


In [60]:
great_table = np.zeros([9, 9], dtype=int)
less_table = np.zeros([9, 9], dtype=int)
count = 0
threshold = 0.2
for index, item in enumerate(uni_scores):
    i = item[0]
    j = item[1]
    review1 = df.iloc[i]
    review2 = df.iloc[j]
    deviation1 = X_all[i]
    deviation2 = X_all[j]
    helpfulness1 = y_all[i]
    helpfulness2 = y_all[j]
    if (deviation1 <= 0 and deviation2 <= 0):
        deviation1 = abs(deviation1)
        deviation2 = abs(deviation2)
        if deviation1 < deviation2:
            if helpfulness1 - helpfulness2 > threshold:
                great_table[int(2*deviation1)][int(2*deviation2)] += 1
            if helpfulness1 - helpfulness2 < -1 * threshold:
                less_table[int(2*deviation1)][int(2*deviation2)] += 1
        if deviation2 < deviation1:
            if helpfulness2 - helpfulness1 > threshold:
                great_table[int(2*deviation2)][[int(2*deviation1)]] += 1
            if helpfulness2 - helpfulness1 < -1 * threshold:
                less_table[int(2*deviation2)][[int(2*deviation1)]] += 1
print (great_table)
print (less_table)

[[0 2 2 0 2 3 0 1 0]
 [0 0 1 2 0 0 0 0 0]
 [0 0 0 2 0 0 1 0 0]
 [0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 2 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]
[[0 0 1 0 1 1 0 0 0]
 [0 0 1 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]


In [61]:
great_table = np.zeros([9, 9], dtype=int)
less_table = np.zeros([9, 9], dtype=int)
count = 0
threshold = 0.2
for index, item in enumerate(uni_scores):
    i = item[0]
    j = item[1]
    review1 = df.iloc[i]
    review2 = df.iloc[j]
    deviation1 = X_all[i]
    deviation2 = X_all[j]
    helpfulness1 = y_all[i]
    helpfulness2 = y_all[j]
    if (deviation1 == -deviation2):
        deviation1 = abs(deviation1)
        deviation2 = abs(deviation2)
        if deviation1 < deviation2:
            if helpfulness1 - helpfulness2 > threshold:
                great_table[int(2*deviation1)][int(2*deviation2)] += 1
            if helpfulness1 - helpfulness2 < -1 * threshold:
                less_table[int(2*deviation1)][int(2*deviation2)] += 1
        if deviation2 < deviation1:
            if helpfulness2 - helpfulness1 > threshold:
                great_table[int(2*deviation2)][[int(2*deviation1)]] += 1
            if helpfulness2 - helpfulness1 < -1 * threshold:
                less_table[int(2*deviation2)][[int(2*deviation1)]] += 1
print (great_table)
print (less_table)

[[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]
